# Creación de al capa OneHotEncoding para modelos de Keras

Vamos a presentar un ejemplo de creación de capa personalizara para el preprocesamiento de la entrada. Concretamente vamos a programar la codificación one-hot como capa que directamente transforma la cadena de texto y devuelve la codificación one-hot de la palabra.

In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df_dinos = pd.DataFrame(data=['diplodocus', 'albertosaurus', 'deinonychus', 'archaeopteryx'], columns=['dinos'])
df_dinos

,dinos
0,diplodocus
1,albertosaurus
2,deinonychus
3,archaeopteryx


El primer paso será convertir las categorías en índices de manera coherente, es decir, que una misma cadena se convierta **siempre** en el mismo índice. Para ello usaremos la capa `TextVectorization` de Keras (de las capas experimentales).

Esta capa se suele usar para estandarizar y tokenizar secuencias de cadenas, como frases, pero en nuestro caso la usaremos únicamente para convertir categorías individuales en índeices de enteros

In [3]:
text_vectorization = tf.keras.layers.experimental.preprocessing.TextVectorization(
    output_sequence_length=1,  # Sólo queremos una única categoría
)
text_vectorization.adapt(
    df_dinos.values,  # Ajusta las categorías para tener siempre las mismas categorías
)

print(f'Vectorized words: {text_vectorization.get_vocabulary()}')
for word in text_vectorization.get_vocabulary():
    print(f'Index for word "{word}": {text_vectorization([[word]])}')

Vectorized words: ['', '[UNK]', 'diplodocus', 'deinonychus', 'archaeopteryx', 'albertosaurus']
Index for word "": [[0]]
Index for word "[UNK]": [[1]]
Index for word "diplodocus": [[2]]
Index for word "deinonychus": [[3]]
Index for word "archaeopteryx": [[4]]
Index for word "albertosaurus": [[5]]


## Creación de la capa personalizada

Ahora crearemos una nueva capa de Keras para que represente una codificación de tipo one-hot en el pipeline. Para ello hay que heredar de [PreprocessingLayer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/PreprocessingLayer)

In [4]:
[i[0] for i in text_vectorization([[v] for v in text_vectorization.get_vocabulary()]).numpy()]

[0, 1, 2, 3, 4, 5]

In [5]:
class OneHotEncodingLayer(tf.keras.layers.experimental.preprocessing.PreprocessingLayer):
    def __init__(self, vocabulary=None, num_categories=None, min_index=None):
        super().__init__()
        self.vectorization = tf.keras.layers.experimental.preprocessing.TextVectorization(
            output_sequence_length=1
        )

        if vocabulary:
            self.vectorization.set_vocabulary(vocabulary)
        self.num_categories = num_categories   
        self.min_index = min_index

    def adapt(self, data):
        self.vectorization.adapt(data)
        vocab = self.vectorization.get_vocabulary()
        self.num_categories = len(vocab)
        indices = [i[0] for i in self.vectorization([[v] for v in vocab]).numpy()]
        self.minimum = min(indices)

    def call(self,inputs):
        vectorized = self.vectorization(inputs)
        subtracted = tf.subtract(vectorized, tf.constant([self.minimum], dtype=tf.int64))
        encoded = tf.one_hot(subtracted, self.num_categories)
        return tf.keras.layers.Reshape((self.num_categories,))(encoded)

    def get_config(self):
        return {
            'vocabulary': self.vectorization.get_vocabulary(),
            'num_categories': self.num_categories,
            'min_index': self.min_index,
        }

El método `adapt`, que viene de la clase `PreprocessingLayer` (el resto son de la clase `Layer`), es llamado **antes** de que comience el entrenamiento. Este es el comportamiento esperado de una capa de preproceso. En dicho paso se establecen los valores de los dos atributos principales de los objetos de la clase:

- `self.num_categories`: El número de categorías únicas entre los datos de entrada
- `self.min_index`: El índice más epqueño producido por la capa `TextVectorization`, que usaremos para restar a sus valores de salida y así que los índices resultantes partan del valor 0

El método `get_config()` permitirá a TensorFlow guardar el estado de la capa cuando el modelo se salva en el disco. Si nos fijamos devuelve los valores del estado, los mismos quepedimos en el inicializador del objeto `__init__()`. La razón es que `adapt` se llama **antes** de que comience el entrenamiento, por lo que cuando se carga la capa de disco, no vamos a llamar a ese método.

## Uso de la capa en nuestros modelos

Tan sencillo como hemos ido usándola hasta ahora:

In [6]:
model_input = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
one_hot_layer = OneHotEncodingLayer()
one_hot_layer.adapt(df_dinos['dinos'].values)
model_output = one_hot_layer(model_input)

model = tf.keras.models.Model(inputs=[model_input], outputs=[model_output])

for dino in df_dinos['dinos']:
    print(f'{dino} -> {model.predict([dino])}')
print(f'UNKNOWN -> {model.predict(["elefante"])}')
print(f'EMPTY_STR -> {model.predict([""])}')

diplodocus -> [[0. 0. 1. 0. 0. 0.]]
albertosaurus -> [[0. 0. 0. 0. 0. 1.]]
deinonychus -> [[0. 0. 0. 1. 0. 0.]]
archaeopteryx -> [[0. 0. 0. 0. 1. 0.]]
UNKNOWN -> [[0. 1. 0. 0. 0. 0.]]
EMPTY_STR -> [[1. 0. 0. 0. 0. 0.]]


Sólo por curiosidad, la clase `StringLookup` (también de `layers.keras.preprocessing`) se puede adaptar para que devuelva una codificación _one-hot_ de una cadena. Sin embargo, este ejemplo sirve para ilustrar cómo se crea una capa personalizada.